## Assumption

- Suppose we have our `Q`, `K`, `V` (query, key, value).
- Now, we want to **calculate the attention** score for each word in the query with respect to each word in the key.

In [1]:
import numpy as np
import math

# L: number of attention heads
# d_k: dimension of keys
# d_v: dimension of values

L, d_k, d_v = 4, 8, 8
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_v)

In [3]:
print("Q\n", q)
print("-------------------------------------")
print("K\n", k)
print("-------------------------------------")
print("V\n", v)

Q
 [[ 0.45788544  1.12479023 -0.23155258  0.73816007  1.44378256 -0.9318273
  -0.94404584 -0.32611646]
 [ 0.86104022  0.64857438 -0.25562526 -1.89072055  0.0527882  -1.33307759
  -0.6267098  -0.43279695]
 [-0.30521636 -0.3914826  -1.19951463 -0.21045987 -0.45477696 -0.137651
   0.22190222  0.27662718]
 [-2.009737   -0.63456588 -0.80733776  2.15382447 -1.16502812  0.48379507
  -2.1392845   1.20571361]]
-------------------------------------
K
 [[ 2.85355768e-01  2.59666731e-01 -9.32228185e-01 -2.71802389e-01
  -7.56521900e-01 -7.82475735e-01  1.04596889e+00 -1.11440436e+00]
 [-4.96510433e-01  9.04111124e-01 -5.66147544e-01  1.57932612e+00
  -1.00602593e+00  5.26469349e-01 -2.70193350e+00  1.03756607e+00]
 [ 2.62923467e-01  2.36562711e-01 -1.69260976e+00  8.03833769e-01
   8.47465689e-02 -5.23533466e-01 -2.48437737e+00 -8.59558944e-02]
 [-8.21725592e-01 -6.39830099e-01 -2.07216735e+00  4.22674674e-01
   1.53798598e+00 -1.44385183e-03  1.02817821e-01 -1.13325799e-01]]
---------------------

---

## Attention Score

$$ attentionScore = softmax\biggl(\frac{(Q . K^T)}{\sqrt d_k} + M \biggr) $$
$$ contextualEmbedding = attentionScore . V $$

Where,
$ d_k = dimension of keys $

$ Q = Query $

$ K = key $

$ V = Value $

$ M = Mask $
(Mask will be useful for decoder)

---

### Why do we need to divide by $\sqrt d_k$?

- It is a **scaling factor** to make the dot product more stable.
- For large values of $d_k$, the dot products grow large in magnitude, pushing the softmax function into regions where it has **extremely small gradients**.
- To counteract this effect, we scale the dot products by $\frac{1}{\sqrt{d_k}}$.
- This helps in **stabilizing the gradients** of the softmax function (more specifically, the softmax function's derivative) making it easier to train.
- This also help prevent the softmax function (that we will apply later on to turn into prob) from **turning probabilities into one-hot (increase prob for element with already high prob and vice-vers)**.

In [4]:
np.matmul(q, k.T)

array([[-0.54918083,  2.35579937,  4.35535453,  1.85764185],
       [ 1.99627671, -2.19313289,  1.58923441, -1.32425516],
       [ 1.36226017,  0.21681912,  1.1467342 ,  2.19015011],
       [-3.64952345, 12.74076461,  7.27843973,  2.49168221]])

In [5]:
# Why we need sqrt(d_k) in denominator
q.var(), k.var(), np.matmul(q, k.T).var()

(0.9747989058894068, 1.0691779403089796, 13.79556247858465)

In [6]:
scaled = np.matmul(q, k.T) / math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(0.9747989058894068, 1.0691779403089796, 1.724445309823081)

In [7]:
scaled

array([[-0.19416474,  0.83290085,  1.53985036,  0.65677557],
       [ 0.7057904 , -0.77538957,  0.56187922, -0.4681949 ],
       [ 0.4816317 ,  0.07665713,  0.40543177,  0.774335  ],
       [-1.29030139,  4.50454053,  2.57331704,  0.88094269]])

---

## Masking 🎭

- This is to ensure words don't get context from words generated in the future.
- Not required in the encoders, but required int he decoders

In [11]:
# make a lower triangular matrix (mask)
mask = np.tril(np.ones((L, L))) 
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [12]:
mask[mask == 0] = -np.infty
mask[mask == 1] = 0

mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

---

- Now, the above mask, when added to any attention score, will make the attention score for the future words as **negative infinity**.
-  This will make the softmax of those scores as **zero**. 
-  This will ensure that the future words don't get any attention from the current word.

---

In [13]:
scaled + mask

array([[-0.19416474,        -inf,        -inf,        -inf],
       [ 0.7057904 , -0.77538957,        -inf,        -inf],
       [ 0.4816317 ,  0.07665713,  0.40543177,        -inf],
       [-1.29030139,  4.50454053,  2.57331704,  0.88094269]])

---

## SoftMax

$$ softmax = \frac{e^{x_i}}{\sum_j e^{x_j}} $$

In [14]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

In [15]:
attention = softmax(scaled + mask)

In [16]:
# attention - you can see that the future tokens are masked
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.81475074, 0.18524926, 0.        , 0.        ],
       [0.3855608 , 0.25716665, 0.35727255, 0.        ],
       [0.00259063, 0.85128087, 0.1234108 , 0.0227177 ]])

---

## New contextual embedding

$$ contextualEmbedding = attentionScore . V $$

In [17]:
new_v = np.matmul(attention, v)
new_v

array([[ 6.75114967e-01, -2.14256015e-01, -1.07847614e+00,
         7.78564027e-01, -9.15212956e-01,  4.21043502e-01,
        -1.02720529e-01,  2.25724423e+00],
       [ 5.53216645e-01,  4.31503817e-04, -8.67278533e-01,
         6.11786763e-01, -3.11615432e-01,  2.58203568e-01,
        -4.27052511e-01,  2.39094460e+00],
       [-2.47219868e-01, -4.85033849e-02, -4.39923118e-01,
         3.07760687e-01, -3.19412647e-01,  3.02474139e-01,
        -6.39875201e-01,  1.91447482e+00],
       [-1.56961633e-01,  7.50696212e-01,  7.28942872e-02,
        -6.16629360e-02,  1.76879273e+00, -3.33547989e-01,
        -1.62201599e+00,  2.69540711e+00]])

In [18]:
v

array([[ 0.67511497, -0.21425601, -1.07847614,  0.77856403, -0.91521296,
         0.4210435 , -0.10272053,  2.25724423],
       [ 0.0170917 ,  0.94465561,  0.06159647, -0.1217217 ,  2.34308633,
        -0.457988  , -1.85350748,  2.97897654],
       [-1.43283647, -0.58450777, -0.11180696,  0.1088236 , -1.59291853,
         0.72190127, -0.34598203,  0.77833161],
       [ 0.15701668,  0.84597437,  1.63090926,  1.1669072 , -1.18310894,
        -1.49014848, -0.05266826,  2.53367017]])

---
